In [1]:
import os
import json
import openai
import requests
from dotenv import load_dotenv
from tenacity import retry, wait_random_exponential, stop_after_attempt


GPT_MODEL = "gpt-4-0613"

load_dotenv(".env")
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, functions=None, model=GPT_MODEL):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + openai.api_key,
    }
    json_data = {"model": model, "messages": messages}
    if functions is not None:
        json_data.update({"functions": functions})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [3]:
class Chat:
    def __init__(self):
        self.conversation_history = []

    def add_prompt(self, role, content):
        message = {"role": role, "content": content}
        self.conversation_history.append(message)

    def display_conversation(self):
        for message in self.conversation_history:
            print(
                f"{message['role']}: {message['content']}\n\n",
                message["role"],
            )


In [4]:
functions = [
    {
        "name": "save_users_questionire",
        "description": "If user responded all questiones, store fully filled questionire to the database",
        "parameters": {
            "type": "object",
            "properties": {
                "user_answers": {
                    "type": "object",
                    "description": "Keys of the dict are questions to the user and values are user's responses to the coresponding questions",
                },
            },
            "required": ["user_answers"],
        },
    },
    {
        "name": "ask_follow_up_question",
        "description": "If the user didn't answer all the questions, generates an additional question to ask user.",
        "parameters": {
            "type": "object",
            "properties": {
                "next_question": {
                    "type": "string",
                    "description": "Next question which we will ask user to clarify their response",
                },
            },
            "required": ["next_question"],
        },
    },
]

In [5]:
chat = Chat()

In [6]:
chat.add_prompt("system", 
                """
                You are a polite and smart AI assistant that helps people to fill questionire.
                We need to fill next questions:
                1) What is your name?
                2) What is your last name?
                3) What is your age?
                We expect final response in json format with keys: name, last_name, age. 
                Age should be int value with year granularity, don't accept a string.
                """)

In [7]:
chat.add_prompt("user", 
               """
               Hi, I'm Bob Smith. I'm looking for a car insurance. Do you have a cheap option?
               
               """)

In [8]:
chat_response = chat_completion_request(
        chat.conversation_history,
        functions=functions
    )


In [9]:
chat_response

<Response [200]>

In [10]:
chat_response.__dict__

{'_content': b'{\n  "id": "chatcmpl-7dCRfV8uD0m2fEzjoIxFVj80yF5f2",\n  "object": "chat.completion",\n  "created": 1689576987,\n  "model": "gpt-4-0613",\n  "choices": [\n    {\n      "index": 0,\n      "message": {\n        "role": "assistant",\n        "content": "Sure, we can definitely help with that. Before we proceed, we need to gather some information. Could you also provide your age, please?"\n      },\n      "finish_reason": "stop"\n    }\n  ],\n  "usage": {\n    "prompt_tokens": 216,\n    "completion_tokens": 30,\n    "total_tokens": 246\n  }\n}\n',
 '_content_consumed': True,
 '_next': None,
 'status_code': 200,
 'headers': {'Date': 'Mon, 17 Jul 2023 06:56:29 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'access-control-allow-origin': '*', 'Cache-Control': 'no-cache, must-revalidate', 'openai-model': 'gpt-4-0613', 'openai-organization': 'user-zqtgiixglanxzg0gvijsewh3', 'openai-processing-ms': '2469', 'openai-version': '20

In [11]:
chat_response._content

b'{\n  "id": "chatcmpl-7dCRfV8uD0m2fEzjoIxFVj80yF5f2",\n  "object": "chat.completion",\n  "created": 1689576987,\n  "model": "gpt-4-0613",\n  "choices": [\n    {\n      "index": 0,\n      "message": {\n        "role": "assistant",\n        "content": "Sure, we can definitely help with that. Before we proceed, we need to gather some information. Could you also provide your age, please?"\n      },\n      "finish_reason": "stop"\n    }\n  ],\n  "usage": {\n    "prompt_tokens": 216,\n    "completion_tokens": 30,\n    "total_tokens": 246\n  }\n}\n'

In [12]:
chat.add_prompt("assistant", 
               """
               Sure, we can definitely help with that. 
               Before we proceed, we need to gather some information. 
               Could you also provide your age, please?
               
               """)

In [13]:
chat.add_prompt("user", 
               """
               I'm under 30.
               
               """)

In [14]:
chat_response = chat_completion_request(
        chat.conversation_history,
        functions=functions
    )

chat_response, chat_response._content

(<Response [200]>,
 b'{\n  "id": "chatcmpl-7dCSHlEH2kiUrkuw2Z8JsUgmGJ9ZC",\n  "object": "chat.completion",\n  "created": 1689577025,\n  "model": "gpt-4-0613",\n  "choices": [\n    {\n      "index": 0,\n      "message": {\n        "role": "assistant",\n        "content": null,\n        "function_call": {\n          "name": "ask_follow_up_question",\n          "arguments": "\\n               {\\n               \\"next_question\\": \\"Could you please specify your exact age?\\"\\n               }"\n        }\n      },\n      "finish_reason": "function_call"\n    }\n  ],\n  "usage": {\n    "prompt_tokens": 271,\n    "completion_tokens": 27,\n    "total_tokens": 298\n  }\n}\n')

In [15]:
chat.add_prompt("assistant", 
               """
               Could you please specify your exact age?
               """)

In [16]:
chat.add_prompt("user", 
               """
               I'm 25.
               
               """)

In [17]:
chat_response = chat_completion_request(
        chat.conversation_history,
        functions=functions
    )

chat_response, chat_response._content

(<Response [200]>,
 b'{\n  "id": "chatcmpl-7dCSzBotn12ZWXceOJC199GcT8MYF",\n  "object": "chat.completion",\n  "created": 1689577069,\n  "model": "gpt-4-0613",\n  "choices": [\n    {\n      "index": 0,\n      "message": {\n        "role": "assistant",\n        "content": null,\n        "function_call": {\n          "name": "save_users_questionire",\n          "arguments": "\\n               {\\n                 \\"name\\": \\"Bob\\",\\n                 \\"last_name\\": \\"Smith\\",\\n                 \\"age\\": 25\\n               }"\n        }\n      },\n      "finish_reason": "function_call"\n    }\n  ],\n  "usage": {\n    "prompt_tokens": 299,\n    "completion_tokens": 35,\n    "total_tokens": 334\n  }\n}\n')